In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [2]:
from loguru import logger
from langchain.callbacks import FileCallbackHandler
from datetime import date

today = date.today()

logfile = f"logs/full_pipeline_{today.year}_{today.month}_{today.day}.log"
logger.add(logfile, format="{message}", colorize=False, enqueue=True)
handler = FileCallbackHandler(logfile)

In [7]:
from langchain.chat_models import ChatOpenAI
cube_selection_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-16k", temperature=0.5, top_p=0.5, model_kwargs={
     "response_format": {
      type: "json_object"
     }
   })

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


In [8]:
await cube_selection_chain.ainvoke({
    "cubes": cubes,
    "question": question,
})

TypeError: keys must be str, int, float, bool or None, not type

In [7]:
from lib import create_cube_selection_chain, fetch_cubes_descriptions, parse_all_cubes, fetch_cube_sample, fetch_dimensions_triplets

cube_selection_settings = {
    "temperature": 0.5,
    "top_p": 0.5
}
cubes = fetch_cubes_descriptions()
cube_selection_chain = create_cube_selection_chain(api_key=OPENAI_API_KEY, handler=handler, **cube_selection_settings)

# question = f"sum of emission of CO2 for industry between year 2009 and 2011"
question = f"average of emission of Methane for transport between years 2007 and 2005"

cube_selection_response1 = await cube_selection_chain.ainvoke({
    "cubes": cubes,
    "question": question,
})
cube_selection_response = cube_selection_response1['text']

logger.info("========== CUBES RESPONSE ================")
logger.info(f"{cube_selection_response}")

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


TypeError: keys must be str, int, float, bool or None, not type

In [ ]:
await generation_chain.ainvoke({
    "cube_and_sample": cube_and_sample,
    "dimensions_triplets": dimensions_triplets,
    "cube": selected_cube,
    "question": question,
})

{'cube_and_sample': '@prefix schema: <http://schema.org/> .\n@prefix qudt: <http://qudt.org/schema/qudt/> .\n@prefix sh: <http://www.w3.org/ns/shacl#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix cube: <https://cube.link/> .\n\n<https://environment.ld.admin.ch/foen/ubd000502/4> a cube:Cube ;\n   cube:observationSet <https://environment.ld.admin.ch/foen/ubd000502/4/observation/> .\n<https://environment.ld.admin.ch/foen/ubd000502/4/observation/> a cube:observationSet ;\n   cube:observation <https://environment.ld.admin.ch/foen/ubd000502/4/observation/123/N2O/2012> .\n<https://environment.ld.admin.ch/foen/ubd000502/4/observation/123/N2O/2012> a cube:Observation ;\n   cube:observedBy <https://ld.admin.ch/office/VII.1.7> ;\n   <https://environment.ld.admin.ch/foen/ubd000502/sektorid> <https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/123> ;\n   <https://environment.ld.admin.ch/fo

In [ ]:
selected_cubes = parse_all_cubes(cube_selection_response)
selected_cube = selected_cubes[0]

logger.info("========== SELECTED FIRST CUBE ================")
logger.info(selected_cube)

2024-01-11 13:05:34.499 | INFO     | __main__:<module>:4 - ========== SELECTED FIRST CUBE ================
2024-01-11 13:05:34.500 | INFO     | __main__:<module>:5 - <https://environment.ld.admin.ch/foen/ubd000502/4>


In [ ]:
cube_and_sample = fetch_cube_sample(selected_cube)

print(cube_and_sample)

@prefix schema: <http://schema.org/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix cube: <https://cube.link/> .

<https://environment.ld.admin.ch/foen/ubd000502/4> a cube:Cube ;
   cube:observationSet <https://environment.ld.admin.ch/foen/ubd000502/4/observation/> .
<https://environment.ld.admin.ch/foen/ubd000502/4/observation/> a cube:observationSet ;
   cube:observation <https://environment.ld.admin.ch/foen/ubd000502/4/observation/123/N2O/2012> .
<https://environment.ld.admin.ch/foen/ubd000502/4/observation/123/N2O/2012> a cube:Observation ;
   cube:observedBy <https://ld.admin.ch/office/VII.1.7> ;
   <https://environment.ld.admin.ch/foen/ubd000502/sektorid> <https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/123> ;
   <https://environment.ld.admin.ch/foen/ubd000502/gas> <https://ld.admin

In [ ]:
dimensions_triplets = fetch_dimensions_triplets(selected_cube)

print(dimensions_triplets)

@prefix schema: <http://schema.org/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix cube: <https://cube.link/> .

<https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/1> schema:name "Switzerland’s greenhouse gas emissions"@en .
<https://ld.admin.ch/cube/dimension/pol01air/pol11> schema:name "Methane"@en .
<https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/2> schema:name "International aviation (not included in total)"@en .
<https://ld.admin.ch/cube/dimension/pol01air/pol9> schema:name "Carbon dioxide"@en .
<https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/3> schema:name "Balance of land use (not included in total)"@en .
<https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/11> schema:name "Buildings"@en .
<https

In [ ]:
from lib import create_query_generation_chain

query_generation_settings = {
    "temperature": 0.15,
    "top_p": 0.1
}

generation_chain = create_query_generation_chain(api_key=OPENAI_API_KEY, handler=handler, **query_generation_settings)

query_generation_response = await generation_chain.ainvoke({
    "cube_and_sample": cube_and_sample,
    "dimensions_triplets": dimensions_triplets,
    "cube": selected_cube,
    "question": question,
})
query_generation_response = query_generation_response['text']

logger.info("========== QUERY GENERATION RESPONSE ================")
logger.info(f"{query_generation_response}")

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


2024-01-11 17:53:10.523 | INFO     | __main__:<module>:18 - ========== QUERY GENERATION RESPONSE ================
2024-01-11 17:53:10.524 | INFO     | __main__:<module>:19 - To answer the question "average of emission of Methane for transport between years 2007 and 2005" for the cube `<https://environment.ld.admin.ch/foen/ubd000502/4>`, we need to modify the query to filter observations by the gas "Methane" and the sector "Transport" for the years 2005 to 2007. We also need to calculate the average of the emissions. Here's how you can modify the query:

```sparql
PREFIX cube: <https://cube.link/>
PREFIX schema: <http://schema.org/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT (AVG(?value) AS ?averageEmission)
WHERE {
    <https://environment.ld.admin.ch/foen/ubd000502/4> a cube:Cube

In [ ]:
from lib import run_query

result = run_query(query_generation_response)

logger.info("=========== QUERY RESULT ============")
logger.info(result)

2024-01-11 13:15:16.575 | INFO     | __main__:<module>:5 - =========== QUERY RESULT ============
2024-01-11 13:15:16.578 | INFO     | __main__:<module>:6 - {'head': {'vars': ['averageEmission']}, 'results': {'bindings': [{'averageEmission': {'datatype': 'http://www.w3.org/2001/XMLSchema#double', 'type': 'literal', 'value': '0.63535825'}}]}}
